In [ ]:
import pandas as pd
person = pd.read_csv('data/hackathon_data/snf/P3_PersonExport.csv', sep=';')

In [ ]:
person

In [ ]:
names = zip(person['First Name'], person['Last Name'])
names_joined = [str(n[0]) + " " + str(n[1]) for n in names]

In [ ]:
import viapy
import viapy.api
import re
import string

viaf = viapy.api.ViafAPI()

In [ ]:
candidates = []

In [ ]:
for name in names_joined[:1000]:
    results = viaf.find_person(name)
    if results != None:
        print(name)
        cands = []
        for r in results:
            if isinstance(r['recordData']['mainHeadings']['data'], list):
                cands.append([x['text'] for x in r['recordData']['mainHeadings']['data']])
            else:
                cands.append(r['recordData']['mainHeadings']['data']['text'])
        candidates.append((name, cands))
    else:
        candidates.append((name, []))

In [112]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [ ]:
for (name, cands) in candidates:
    if len(cands) > 0:
        print(name, cands)
        ld = 1000
        best_match = None
        for persons in cands:
            for p in persons:
                p_cleaned = re.sub(r'\d+', '', p).translate(str.maketrans('', '', string.punctuation))
                d = levenshteinDistance(name, p_cleaned)
                if d < ld:
                    ld = d
                    best_match = p_cleaned
        print(best_match, ld)